In [ ]:
import ibmos2spark, os
# @hidden_cell

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_1 = '#################################'
else:
    endpoint_1 = '#################################'

credentials = {
    'endpoint': endpoint_1,
    'service_id': '##############################',
    'iam_service_endpoint': '##############################',
    'api_key': '##############################'
}

configuration_name = '###################################'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
agg_base = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('agg_base.csv', '###################################'))

In [ ]:
df_01 = agg_base.withColumn('VrTotalVendido_12m', col('VrTotalVendido_12m').cast('double'))\
                .withColumn('QtTotalSkus_12m', col('QtTotalSkus_12m').cast('integer'))\
                .withColumn('avg_TxCalculoNPS', col('avg_TxCalculoNPS').cast('double'))\
                .withColumn('porcentagempositivo', col('porcentagempositivo').cast('integer'))\
                .withColumn('QtTotalCompraEntregaENT_12m', col('QtTotalCompraEntregaENT_12m').cast('integer'))\
                .withColumn('VrTotalVendido_9m', col('VrTotalVendido_9m').cast('double'))\
                .withColumn('QtTotalSkus_9m', col('QtTotalSkus_9m').cast('integer'))\
                .withColumn('avg_TxCalculoNPS', col('avg_TxCalculoNPS').cast('double'))\
                .withColumn('porcentagempositivo', col('porcentagempositivo').cast('integer'))\
                .withColumn('QtTotalCompraEntregaENT_9m', col('QtTotalCompraEntregaENT_9m').cast('integer'))\
                .withColumn('VrTotalVendido_6m', col('VrTotalVendido_6m').cast('double'))\
                .withColumn('QtTotalSkus_6m', col('QtTotalSkus_6m').cast('integer'))\
                .withColumn('avg_TxCalculoNPS', col('avg_TxCalculoNPS').cast('double'))\
                .withColumn('porcentagempositivo', col('porcentagempositivo').cast('integer'))\
                .withColumn('QtTotalCompraEntregaENT_6m', col('QtTotalCompraEntregaENT_6m').cast('integer'))\
                .withColumn('VrTotalVendido_3m', col('VrTotalVendido_3m').cast('double'))\
                .withColumn('QtTotalSkus_3m', col('QtTotalSkus_3m').cast('integer'))\
                .withColumn('avg_TxCalculoNPS', col('avg_TxCalculoNPS').cast('double'))\
                .withColumn('porcentagempositivo', col('porcentagempositivo').cast('integer'))\
                .withColumn('QtTotalCompraEntregaENT_3m', col('QtTotalCompraEntregaENT_3m').cast('integer'))

In [ ]:
def score_v0(df, months):
    skus_col = 'QtTotalSkus_'+months+'m'
    newskus_col = 'skus_score_'+months+'m'
    ent_col = 'QtTotalCompraEntregaENT_'+months+'m'
    newent_col = 'entregues_score_'+months+'m'
    faturamento_col = 'VrTotalVendido_'+months+'m'
    newfaturamento_col = 'faturamento_score_'+months+'m'
    
    
    df_final = df.withColumn(newskus_col, when(col(skus_col).between(1, 5), lit('0.10'))\
                         .when(col(skus_col).between(5, 10), lit('0.20'))\
                         .when(col(skus_col).between(10, 150), lit('0.50'))\
                         .when(col(skus_col).between(150,500), lit('0.70'))\
                         .when(col(skus_col)>=(500), lit('1')).otherwise('0.01'))\
                .withColumn(newfaturamento_col, when(col(faturamento_col).between(100, 500), lit('0.10'))\
                         .when(col(faturamento_col).between(500, 1500), lit('0.20'))\
                         .when(col(faturamento_col).between(1500, 5000), lit('0.50'))\
                         .when(col(faturamento_col).between(5000, 10000), lit('0.70'))\
                         .when(col(faturamento_col)>=(10000), lit('1')).otherwise('0.010'))\
                .withColumn(newent_col, when(col(ent_col).between(1, 5), lit('0.10'))\
                         .when(col(ent_col).between(5, 10), lit('0.20'))\
                         .when(col(ent_col).between(10, 150), lit('0.50'))\
                         .when(col(ent_col).between(150,500), lit('0.70'))\
                         .when(col(ent_col)>=(500), lit('1')).otherwise('0.01'))\
                .withColumn(f'score_vendas_{months}m', (4*col(newskus_col) + 4*col(newfaturamento_col) + 2*col(newent_col))/10)\
                .withColumn(f'score_atendimento_{months}m', ((2*col('avg_TxCalculoNPS')/100) + (2*col('porcentagempositivo')/100))/4)\
                .withColumn(f'score_lojista_{months}m', (2*col(f'score_atendimento_{months}m')+8*col(f'score_vendas_{months}m'))/10 )
    return df_final

In [ ]:
df_12 = score_v0(df_01, months='12')
df_9 = score_v0(df_12, months='9')
df_6 = score_v0(df_9, months='6')
df_final = score_v0(df_6, months='3')

In [ ]:
pd_final_df = df_final.toPandas()

In [ ]:
from project_lib import Project
project = Project(sc,"#################################","################################")
project.save_data(file_name = "score_v0.json", data = pd_final_df.to_json(orient='records'))

In [ ]:
from project_lib import Project
project = Project(sc,"##################################","#################################")
project.save_data(file_name = "score_v0.csv", data = pd_final_df.to_csv(index=False))